# Determine volume of bound electrons - density extraction

Preparation of data, such that only density in spheres around nuclei is used for calculation of atomic energies

strategy: set density out side of radii to zero, then these gridpoints do not contribute to the integral $\int d\vec{r} \frac{\tilde{\rho}}{|\vec{r}-\vec{R_I}|}$

this way after loading the density files; an additional step is carried out as implemented in this notebook
in this step the density array is changed as described above; then the atomic energies can be calculated as already implemented without any changes to the existing code

In [85]:
import numpy as np
import scipy.spatial as scsp

import sys
sys.path.insert(0, '/home/misa/APDFT/prototyping/atomic_energies/')

import parse_density_files as pdf

In [86]:
def meshgrid2vector(grid):
    """
    convert components of meshgrid into set of vectors; every vector is a point of the grid
    e.g. a 3D grid is converted in a numpy array of shape (number gridpoints, 3)
    """
    flattened_grid = []
    for c in grid:
        flattened_grid.append(c.flatten())
        
    return(np.array(flattened_grid).T)
    

# get grid points lying in sphere with radius r_vdW

def get_idc_rvdW(center, rvdw, gpts):
    """
    returns indices of grid points lying in a sphere around center with radius rdwV
    
    center: center of sphere
    rdvw: radius of sphere
    coordinates of grid as flattened numpy array, shape (number of grid points, dimension of grid)
    """
    
    # distance of every gridpoint from center
    distance_array = scsp.distance.cdist(gpts, center)
    
    
    idc_in_sphere = np.where(distance_array[:, 0] <= rvdw)[0] # indices of points within sphere with vdW radius
    idc_out_sphere = np.where(distance_array[:, 0] > rvdw)[0] # indices of points out of sphere with vdW radius
    
    return(idc_in_sphere, idc_out_sphere)

In [87]:
# load data

In [88]:
path = '/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/dsgdb9nsd_002626/cube-files/ve_8.cube'
cube = pdf.CUBE(path)

In [89]:
# prepare data

In [90]:
grid = cube.get_grid()
density = cube.data_scaled
np.amax(density)

0.0004897465005113976

In [91]:
nucleus = np.array([[10.0, 10.0, 10.0]])
radius = 5.0

nucleus2 = np.array([[20.0, 10.0, 10.0]])
radius2 = 7.0

In [92]:
# get indices within and out of radius around nuclei

In [93]:
idc_in, idc_out = get_idc_rvdW(nucleus, radius, meshgrid2vector(grid))
idc_in2, idc_out2 = get_idc_rvdW(nucleus2, radius2, meshgrid2vector(grid))

In [94]:
idc_in

array([ 742219,  742220,  742221, ..., 2121921, 2121922, 2121923])

In [95]:
# select all indices that are within at least one radius for one nucleus

In [96]:
print(idc_in.shape)
print(idc_in2.shape)
print(idc_in.shape[0]+idc_in2.shape[0])
print(np.concatenate((idc_in, idc_in2)).shape)

(51978,)
(142632,)
194610
(194610,)


In [97]:
all_idc_in = np.concatenate((idc_in, idc_in2))
all_idc_in = np.unique(all_idc_in)
print(all_idc_in.shape)

(191186,)


In [98]:
# select all indices that are NOT within at least one radius for one nucleus

In [99]:
all_idc_out = np.setdiff1d(np.arange(np.prod(density.shape)), all_idc_in, assume_unique=True)

In [100]:
# set density around nuclei to zero

In [101]:
density_flattened = density.flatten()
density_flattened[all_idc_out] = 0.0

In [102]:
density = density_flattened.reshape(density.shape)
np.amax(density)

1.1308351258260211e-05

# Some tests

In [ ]:
import numpy as np
import scipy.spatial as scsp

import sys
sys.path.insert(0, '/home/misa/APDFT/prototyping/atomic_energies/')

import parse_density_files as pdf

In [ ]:
# import test data

In [ ]:
path = '/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/dsgdb9nsd_002626/cube-files/ve_8.cube'
cube = pdf.CUBE(path)

In [ ]:
# calculate distance between center point in cell and gridpoints

In [ ]:
grid = cube.get_grid()
x = grid[0].flatten()
y = grid[1].flatten()
z = grid[2].flatten()

distance_array = scsp.distance.cdist(np.array([x,y,z]).T, np.array([[10,10,10]]))

In [ ]:
# select points within certain distance from center point

In [ ]:
np.where(distance_array[:,0] <= 5)[0]

In [ ]:
distance_array[:,0]

In [ ]:
np.where(distance_array <= 5)[0].shape

In [ ]:
np.where(distance_array <= 5)

In [ ]:
# make sure that shapes of inputs are equal

In [ ]:
np.array([[10,10,10]]).shape # shape center point

In [ ]:
np.array([x,y,z]).T.shape # shape grid

In [ ]:
scsp.distance.cdist(np.array([x,y,z]).T, np.array([[10,10,10]])).shape

In [ ]:
# Test if correct distance

In [ ]:
np.linalg.norm(np.array([37.578606, 37.578606, 37.578606]) - np.array([[10,10,10]]))

In [ ]:
np.sqrt(3*27**2)

In [ ]:
def meshgrid2vector(grid):
    """
    convert components of meshgrid into set of vectors; every vector is a point of the grid
    e.g. a 3D grid is converted in a numpy array of shape (number gridpoints, 3)
    """
    flattened_grid = []
    for c in grid:
        flattened_grid.append(c.flatten())
        
    return(np.array(flattened_grid).T)
    

# get grid points lying in sphere with radius r_vdW

def get_idc_rvdW(center, rvdw, gpts):
    """
    returns indices of grid points lying in a sphere around center with radius rdwV
    
    center: center of sphere
    rdvw: radius of sphere
    coordinates of grid as flattened numpy array, shape (number of grid points, dimension of grid)
    """
    
    # distance of every gridpoint from center
    distance_array = scsp.distance.cdist(gpts, center)
    
    
    idc_in_sphere = np.where(distance_array[:, 0] <= rvdw) # indices of points within sphere with vdW radius
    idc_out_sphere = np.where(distance_array[:, 0] > rvdw) # indices of points out of sphere with vdW radius
    
    return(idc_in_sphere, idc_out_sphere)
    

In [ ]:
# set density for points out of distance to center point to zero

In [ ]:
density_flattened = np.reshape(cube.data, 175**3)
idc_in, idc_out = get_idc_rvdW(np.array([[10,10,10]]), 5.0, meshgrid2vector(grid))
density_flattened[idc_out] = 0.0

In [ ]:
density_flattened.reshape((175, 175, 175))

In [ ]:
# make sure that reshaping is fine

In [ ]:
np.array_equal(np.array([np.reshape(grid[0], 175**3), np.reshape(grid[1], 175**3), np.reshape(grid[2], 175**3)]).T, meshgrid2vector(grid))

In [ ]:
np.array_equal(meshgrid2vector(grid),np.array([x,y,z]).T)